In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from os import walk

In [97]:
filenames = next(walk('/home/clement/code/ssaulay/silentspeak/drafts/data/patient0'), (None, None, []))[2]
for filename in filenames:
    print(filename)

35_front - 0.avi
14_front - 0.avi
28_front - 0.avi
33_front - 0.avi
36_front - 0.avi
37_front - 0.avi
30_front - 0.avi
26_front - 0.avi
32_front - 0.avi


In [98]:
filenames = next(walk('/home/clement/code/ssaulay/silentspeak/drafts/data/patient0'), (None, None, []))[2]
name_csv = []

for filename in filenames:
            
    path = f"/home/clement/code/ssaulay/silentspeak/drafts/data/patient0/{filename}"
    # path = '/home/clement/code/ssaulay/silentspeak/patient0/14_front - 0.avi'
    cap = cv2.VideoCapture(path)
    NUM_FACE = 2
    mp_face_mesh = mp.solutions.face_mesh
    mp_drawing = mp.solutions.drawing_utils
    faceMesh = mp_face_mesh.FaceMesh(max_num_faces=NUM_FACE)
    drawSpec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

    coordinates_by_frames = []
    with mp_face_mesh.FaceMesh() as face_mesh:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            
            # Convert the BGR frame to RGB and process it
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Process the frame using MediaPipe
            results = face_mesh.process(rgb_frame)  

            # Draw the face landmarks on the frame
            annotated_frame = frame.copy()
            
            if results.multi_face_landmarks:
                for faceLms in results.multi_face_landmarks:
                    mp_drawing.draw_landmarks(
                        annotated_frame, 
                        results.multi_face_landmarks[0], 
                        mp_face_mesh.FACEMESH_CONTOURS,
                        drawSpec, 
                        drawSpec
                        )
            
            # Extract the mouth region
            mouth_coordinates = {}
            mouth_landmarks = []
            mouth_ids = [
            0, 267, 269, 270, 409, 291, 375, 321, 405, 314,
            17, 84, 181, 91, 146, 61, 185, 40, 39, 37, 0,
            13, 312, 311, 310, 415, 308, 324, 318, 402, 317,
            14, 87, 178, 88, 95, 78, 191, 80, 81, 82, 13
            ]
            reference_id = 6
            
            if results.multi_face_landmarks:
                landmarks = results.multi_face_landmarks[0]
                
                for id, landmark in enumerate(landmarks.landmark):
                    
                    # Extract only the mouth landmarks (IDs in mouth_ids)
                    if id in mouth_ids:
                        # Get the frame, id and coordinates of points of mouth
                        pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
                        coordinates_by_frames.append(pos_frame)
                        coordinates_by_frames.append(id)
                        coordinates_by_frames.append(landmark.x)
                        coordinates_by_frames.append(landmark.y)
                        coordinates_by_frames.append(landmark.z)
                        ih, iw, ip = frame.shape
                        x, y, z= int(landmark.x * iw), int(landmark.y * ih), int(landmark.y * ip)
                        mouth_landmarks.append((x, y))
                        mouth_coordinates[id] = landmarks
                        cv2.putText(frame, str(id), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1)
                        
                    if id == reference_id:
                        # Get the frame, id and coordinates of reference point
                        pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
                        coordinates_by_frames.append(pos_frame)
                        coordinates_by_frames.append(id)
                        coordinates_by_frames.append(landmark.x)
                        coordinates_by_frames.append(landmark.y)
                        coordinates_by_frames.append(landmark.z)

            # Draw the mouth region
            if mouth_landmarks:
                hull = cv2.convexHull(np.array(mouth_landmarks))
                cv2.drawContours(annotated_frame, [hull], -1, (0, 255, 0), 2)
                    
            # Identify mouth using haar-based classifiers
            # mouth_cascade = cv2.CascadeClassifier('/home/clement/code/SimpleCV/SimpleCV/Features/HaarCascades/face.xml')
            # mouth = mouth_cascade.detectMultiScale(rgb_frame, 1.5, 11)
            # for(mx, my, mw, mh) in mouth:
            #     cv2.rectangle(annotated_frame, (mx, my), (mx+mw, my+mh), (255, 0, 0), 2)
            
            # Show the video
            # cv2.imshow('Mouth Capture', annotated_frame)


            # Press 'q' to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
        
    # Create dataframe from array
    coordinates_by_frames_array = [np.array(coordinates_by_frames[i:i+5]) for i in range(0, len(coordinates_by_frames), 5)]
    df = pd.DataFrame(coordinates_by_frames_array)
    df.rename(columns={0:'frame', 1:'point', 2:'x', 3:'y', 4:'z'}, inplace=True)
    
    path_csv = "/home/clement/code/ssaulay/silentspeak/drafts/data/csv/"
    # Create csv non-scale
    name_csv = path_csv+filename[:-4]+".csv"
    df.to_csv(name_csv, index=False)

    cap.release()
    cv2.destroyAllWindows()

In [59]:
df

,frame,point,x,y,z
0,1.0,0.0,0.558585,0.791090,-0.018064
1,1.0,6.0,0.560718,0.692983,-0.022962
2,1.0,13.0,0.558631,0.799808,-0.011355
3,1.0,14.0,0.557159,0.820927,-0.004900
4,1.0,17.0,0.557139,0.836300,-0.004483
...,...,...,...,...,...
1676,41.0,375.0,0.587777,0.803062,0.005318
1677,41.0,402.0,0.571564,0.813304,-0.001386
1678,41.0,405.0,0.576389,0.824508,-0.001042
1679,41.0,409.0,0.587876,0.785719,-0.000217


In [88]:
# Create dataframe from array
coordinates_by_frames_array = [np.array(coordinates_by_frames[i:i+5]) for i in range(0, len(coordinates_by_frames), 5)]
df = pd.DataFrame(coordinates_by_frames_array)
df.rename(columns={0:'frame', 1:'point', 2:'x', 3:'y', 4:'z'}, inplace=True)

for name in name_csv:
    df.to_csv('name', index=False)
df[df['frame']==1]

,frame,point,x,y,z
0,1.0,0.0,0.558585,0.791090,-0.018064
1,1.0,6.0,0.560718,0.692983,-0.022962
2,1.0,13.0,0.558631,0.799808,-0.011355
3,1.0,14.0,0.557159,0.820927,-0.004900
4,1.0,17.0,0.557139,0.836300,-0.004483
5,1.0,37.0,0.548611,0.788965,-0.017055
6,1.0,39.0,0.538812,0.790925,-0.012283
7,1.0,40.0,0.532893,0.794083,-0.005717
8,1.0,61.0,0.525251,0.801290,0.008676
9,1.0,78.0,0.530336,0.801356,0.006501


In [89]:
df

,frame,point,x,y,z
0,1.0,0.0,0.558585,0.791090,-0.018064
1,1.0,6.0,0.560718,0.692983,-0.022962
2,1.0,13.0,0.558631,0.799808,-0.011355
3,1.0,14.0,0.557159,0.820927,-0.004900
4,1.0,17.0,0.557139,0.836300,-0.004483
...,...,...,...,...,...
1676,41.0,375.0,0.587777,0.803062,0.005318
1677,41.0,402.0,0.571564,0.813304,-0.001386
1678,41.0,405.0,0.576389,0.824508,-0.001042
1679,41.0,409.0,0.587876,0.785719,-0.000217


In [90]:
# Copy dataframe
df_scale = df

for i in range(0, df_scale.loc[df_scale['point'] == 6, 'x'].count()):
    print(df_scale.loc[df_scale['point'] == 6, 'x'].values[i])

0.5607177019119263
0.5608675479888916
0.5611159801483154
0.5619637966156006
0.5618023872375488
0.5617461204528809
0.5613106489181519
0.5610095858573914
0.5610891580581665
0.5606311559677124
0.5608748197555542
0.5604379773139954
0.5616393089294434
0.561854362487793
0.56178218126297
0.5621093511581421
0.5621150135993958
0.5619669556617737
0.5617807507514954
0.5617486238479614
0.5614659786224365
0.5608965754508972
0.5610900521278381
0.5608353614807129
0.560332179069519
0.5601416230201721
0.5595857501029968
0.5588624477386475
0.5581344962120056
0.5578153133392334
0.5568233132362366
0.5558971762657166
0.5555977821350098
0.5553333759307861
0.5550171136856079
0.5548707246780396
0.5548214912414551
0.5546402931213379
0.5548869967460632
0.5550336241722107
0.5549790263175964


In [72]:
# Copy dataframe
df_scale = df

# Get reference coordinates for ID 6 (only frame 1)
ref_x = df_scale.loc[df_scale['point'] == 6, 'x'].values[0]
ref_y = df_scale.loc[df_scale['point'] == 6, 'y'].values[0]
ref_z = df_scale.loc[df_scale['point'] == 6, 'z'].values[0]

# Compute difference between coordinates
df_scale['x'] = df_scale['x'] - ref_x
df_scale['y'] = df_scale['y'] - ref_y
df_scale['z'] = df_scale['z'] - ref_z

df_scale[df_scale['frame']==1]

,frame,point,x,y,z
0,1.0,0.0,-0.002132,0.098107,0.004898
1,1.0,6.0,0.000000,0.000000,0.000000
2,1.0,13.0,-0.002087,0.106825,0.011607
3,1.0,14.0,-0.003559,0.127945,0.018063
4,1.0,17.0,-0.003579,0.143317,0.018479
5,1.0,37.0,-0.012107,0.095982,0.005908
6,1.0,39.0,-0.021906,0.097942,0.010679
7,1.0,40.0,-0.027825,0.101100,0.017245
8,1.0,61.0,-0.035466,0.108307,0.031639
9,1.0,78.0,-0.030381,0.108373,0.029463


In [80]:
df_scale[df_scale['frame']==2]

,frame,point,x,y,z
41,2.0,0.0,-0.001534,0.095619,0.005532
42,2.0,6.0,0.000150,-0.000901,-0.000672
43,2.0,13.0,-0.001597,0.104291,0.012234
44,2.0,14.0,-0.002110,0.127968,0.019749
45,2.0,17.0,-0.002234,0.143736,0.020323
46,2.0,37.0,-0.011961,0.093691,0.006505
47,2.0,39.0,-0.022140,0.095710,0.011426
48,2.0,40.0,-0.028060,0.098820,0.018588
49,2.0,61.0,-0.035582,0.106593,0.033805
50,2.0,78.0,-0.030271,0.106456,0.031686
